https://towardsdatascience.com/predicting-the-music-mood-of-a-song-with-deep-learning-c3ac2b45229e

https://realpython.com/python-nltk-sentiment-analysis/

MEDIUM : https://towardsdatascience.com/how-to-analyze-emotions-and-words-of-the-lyrics-from-your-favorite-music-artist-bbca10411283

In [3]:
#libraries used to extract, clean and manipulate the datafrom helpers import *

import pandas as pd
import numpy as np
import string#To plot the graphsfrom wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn')#library used to count the frequency of wordsfrom sklearn.feature_extraction.text import CountVectorizer#To create the sentiment analysis model, tokenization and lemmatizationimport nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
import nltk.data
nltk.download('vader_lexicon')
nltk.download('punkt')

import time

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#Spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

spotify_client_id = "3a2f65eb20164b24b59b7fdd6876bb51"
spotify_client_secret = "9f6d3a070cdf4eb8adda14a794d60469"

#Genius credentials
genius_token = "4SoLPv5UHyFZyAe5T-PhxS-bRCRrdNIgMNpeqjjIaWvnRbUmTpoGxd9knxBtlQyg"

#Authentication spotify - without user
client_credentials_manager = SpotifyClientCredentials(client_id="3a2f65eb20164b24b59b7fdd6876bb51", client_secret="9f6d3a070cdf4eb8adda14a794d60469")
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
#playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbL3J0k32lWnN?si=5c3f6128e25049d9"  #top50DK
playlist_link ="https://open.spotify.com/playlist/4XDFklOPJOJp3geewWo6vo?si=97b4324becef4027"  #Small 6 song playlist
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [4]:
for track in sp.playlist_tracks(playlist_URI)["items"]:
    
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    
    #Album
    album = track["track"]["album"]["name"]
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]
    
    #Year
    track_year = track["track"]["album"]["release_date"]
    
    print(track_year)
    #Printin details
    #print(track_name,"\n", 
    #      artist_name,"\n",
    #      #sp.audio_features(track_uri)[0], "\n", 
    #      track_uri
    #     )

2019-06-28
2020-04-03
2021-05-14
2019-07-19
2021-08-16


In [5]:
sp.audio_features("https://open.spotify.com/track/7MXVkk9YMctZqd1Srtv4MB?si=e11b8d5376554fde")[0]

{'danceability': 0.679,
 'energy': 0.587,
 'key': 7,
 'loudness': -7.015,
 'mode': 1,
 'speechiness': 0.276,
 'acousticness': 0.141,
 'instrumentalness': 6.35e-06,
 'liveness': 0.137,
 'valence': 0.486,
 'tempo': 186.003,
 'type': 'audio_features',
 'id': '7MXVkk9YMctZqd1Srtv4MB',
 'uri': 'spotify:track:7MXVkk9YMctZqd1Srtv4MB',
 'track_href': 'https://api.spotify.com/v1/tracks/7MXVkk9YMctZqd1Srtv4MB',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7MXVkk9YMctZqd1Srtv4MB',
 'duration_ms': 230453,
 'time_signature': 4}

In [177]:
data = pd.read_csv("../data/cleanedDatasetWithFeatures.csv")

In [126]:
data.head()

,Unnamed: 0.1,Unnamed: 0,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,253019.0,0.852,0.773,8.0,-2.921,0.0776,0.187,0.000030,0.1590,0.9070,102.034,195840.0
1,1,10123748,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,0.1700,0.209,0.000000,0.1120,0.8460,177.833,228200.0
2,2,1305825,ภาพจำ,1,2018-04-12,Pop Pongkool,https://open.spotify.com/track/4YRpjJaJpiI36HM...,Thailand,15477.0,0.557,0.585,5.0,-7.559,0.0470,0.426,0.000001,0.1440,0.4640,81.922,274000.0
3,3,2943946,Te Boté - Remix,1,2018-04-27,"Nio Garcia, Casper Magico, Bad Bunny, Darell, ...",https://open.spotify.com/track/3V8UKqhEK5zBkBb...,Honduras,12869.0,0.903,0.675,11.0,-3.445,0.2140,0.542,0.000013,0.0595,0.4420,96.507,417920.0
4,4,19131998,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,0.0320,0.423,0.000002,0.4350,0.0979,129.863,221538.0


## Adding features of each song to the dataset

In [10]:
test_set = data.head()

In [11]:
test_set.shape

(5, 20)

In [ ]:
# Add columns to existing dataset with the audio features.
# Implementing a timer to check the time for adding new columns.
import time
tic = time.perf_counter()

errors = 0

for index, row in data.iterrows():
    try:
        current = sp.audio_features(row['url'])[0]
        data.at[index,'danceability'] = float(current['danceability'])
        data.at[index,'energy'] = current['energy']
        data.at[index,'key'] = current['key']
        data.at[index,'loudness'] = current['loudness']
        data.at[index,'mode'] = current['mode']
        data.at[index,'speechiness'] = current['speechiness']
        data.at[index,'acousticness'] = current['acousticness']
        data.at[index,'instrumentalness'] = current['instrumentalness']
        data.at[index,'liveness'] = current['liveness']
        data.at[index,'valence'] = current['valence']
        data.at[index,'tempo'] = current['tempo']
        data.at[index,'duration_ms'] = current['duration_ms']
        print(str(int(((index+1)/data.shape[0])*100)) + '% done - ' + str(index+1) + ' of ' + str(data.shape[0]) + ' songs iterated - ' + errors + ' encountered.', end='\r')
    except:
        #append error or insert specific values?
        errors = errors+1
        pass
    
        
toc = time.perf_counter()
print(f"Added new rows from external links in {toc - tic:0.4f} seconds")

In [ ]:
data.isnull().sum()

**Missing values from iteration:**

In [ ]:
'''
6119
20101
23710 
25962
33986
34521 
38726 
39988 
52686
63662 
64375
66721
72738
77099 
94502 
96669 
99117 
'''

In [ ]:
data[6118:6120]

In [ ]:
#Saving the sub dataset for testing.
from pathlib import Path
path = Path('../data/cleanedDatasetWithFeatures.csv')
path.parent.mkdir(parents=True, exist_ok=True)
data.to_csv(path)

In [ ]:
new_data = data.head()

In [ ]:
new_data.head()

## Fetching genres from genius

In [34]:
import csv
import requests
import json
import re
import time
from pandas.io.json import json_normalize
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [35]:
data = pd.read_csv("../data/cleanedDatasetWithFeatures.csv")

In [36]:
def get_track_name(nr):
    track_string = str(data.title[nr])
    x = track_string.find(" (")
    y = track_string.find(" -")
    
    if(int(x) < 0):
        track_string = track_string
    if(int(x) > 0):
        track_string = track_string[:x]
    if (int(y) > 0):
        track_string = track_string[:y]
    
    return track_string

def get_track_artist(nr):
    
    artist_string = str(data.artist[nr])

    y = artist_string.find(", ")
    x = artist_string.find(" (")
    
    if(int(x) > 0):
        artist_string = artist_string.replace("' ", "")

    if(int(y) > 0):
        #artist_string = y
        artist_string = artist_string[:y]     
        
    return artist_string

In [37]:
def get_url(track_nr):
    track = re.sub('[^a-zA-Z0-9 \n\.]', '', str(get_track_name(track_nr)))
    artist = re.sub('[^a-zA-Z0-9 \n\.]', '', str(get_track_artist(track_nr)))
    
    if (int(artist.find(" ")) > 0):
        artist = artist.replace(" ", "-")
        
    if (int(track.find(" ")) > 0):
        track = track.replace(" ", "-")
    
    url_string = str('https://genius.com/' + artist + '-' + track + '-lyrics')
    
    return str(url_string)

In [44]:
def extract_credits(URL):
        
    r = requests.get(URL)
    
    if (int(r.status_code) == 200):
        
        genre = []

        # Parsing the HTML
        soup = BeautifulSoup(r.content, 'html.parser')
        g = soup.find(class_='SongTags__Container-xixwg3-1 bZsZHM')
        ggenre = g.find('a')        
                    
    else:
        return ''
        
        
    return ggenre.text

In [45]:
extract_credits(get_url(data.iloc[50][0]))

'Rap'

In [46]:
get_url(data.iloc[5][0])

'https://genius.com/Ariana-Grande-7-rings-lyrics'

In [59]:
df_with_genre.drop('Unnamed: 0', inplace=True, axis=1)
df_with_genre.drop('Unnamed: 0.1', inplace=True, axis=1)

C:\Users\marti\AppData\Local\Temp\ipykernel_10168\291190469.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_genre.drop('Unnamed: 0', inplace=True, axis=1)
C:\Users\marti\AppData\Local\Temp\ipykernel_10168\291190469.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_genre.drop('Unnamed: 0.1', inplace=True, axis=1)


In [41]:
sub_data = data[:100]

In [47]:
# Add genres to the existing dataset.

tic = time.perf_counter()

errors = 0

for index, row in data.iterrows():
    print(str(int(((index+1)/data.shape[0])*100)) + '% done - ' + str(index+1) + ' of ' + str(data.shape[0]) + ' songs iterated - ' + str(errors) + ' null values encountered.', end='\r')
    data.at[index,'genre'] = extract_credits(get_url(index))

toc = time.perf_counter()
print(f"Added new rows from external links in {toc - tic:0.4f} seconds")

KeyboardInterrupt: 

In [48]:
# Check how many values are null.
data['genre'].isnull().sum()

89516

In [50]:
df_with_genre = data[data['genre'].notna()]

In [51]:
df_with_genre

,Unnamed: 0.1,Unnamed: 0,title,rank,date,artist,url,region,streams,danceability,...,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genre
0,0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,253019.0,0.852,...,8.0,-2.921,0.0776,0.1870,0.000030,0.1590,0.9070,102.034,195840.0,Pop
1,1,10123748,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,...,2.0,-4.757,0.1700,0.2090,0.000000,0.1120,0.8460,177.833,228200.0,Pop
2,2,1305825,ภาพจำ,1,2018-04-12,Pop Pongkool,https://open.spotify.com/track/4YRpjJaJpiI36HM...,Thailand,15477.0,0.557,...,5.0,-7.559,0.0470,0.4260,0.000001,0.1440,0.4640,81.922,274000.0,
3,3,2943946,Te Boté - Remix,1,2018-04-27,"Nio Garcia, Casper Magico, Bad Bunny, Darell, ...",https://open.spotify.com/track/3V8UKqhEK5zBkBb...,Honduras,12869.0,0.903,...,11.0,-3.445,0.2140,0.5420,0.000013,0.0595,0.4420,96.507,417920.0,
4,4,19131998,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,...,5.0,-10.606,0.0320,0.4230,0.000002,0.4350,0.0979,129.863,221538.0,Pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15040,15040,1958808,Cesarz,14,2018-03-26,KęKę,https://open.spotify.com/track/0lQffc27GaikHNY...,Poland,26276.0,0.762,...,7.0,-6.824,0.2670,0.1250,0.000000,0.1960,0.5810,130.101,189986.0,
15041,15041,8709309,Someday,14,2019-08-27,Michele 99,https://open.spotify.com/track/6Yb2WjAUX3bs9dy...,Nicaragua,7274.0,0.577,...,9.0,-6.608,0.1590,0.0132,0.000000,0.2650,0.5200,79.979,192026.0,
15042,15042,1582372,Højre Og Venstre (feat. Ericka Jane),14,2018-06-01,Skinz,https://open.spotify.com/track/610KxSt2HNjczZ2...,Denmark,45927.0,0.521,...,10.0,-7.384,0.1490,0.2190,0.000000,0.2300,0.3650,82.572,173912.0,
15043,15043,20567050,Addict Of Magic,14,2021-06-25,Picture This,https://open.spotify.com/track/0deLQzMIEw6l3pW...,Ireland,31208.0,0.739,...,0.0,-7.555,0.0269,0.5840,0.000000,0.1530,0.6030,88.017,166897.0,Pop


In [52]:
df_with_genre = df_with_genre[df_with_genre['genre'] != '']

In [62]:
df_with_genre.reset_index()

,index,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,genre
0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,253019.0,0.852,0.773,8.0,-2.921,0.0776,0.18700,0.000030,0.1590,0.9070,102.034,195840.0,Pop
1,1,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,0.1700,0.20900,0.000000,0.1120,0.8460,177.833,228200.0,Pop
2,4,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,0.0320,0.42300,0.000002,0.4350,0.0979,129.863,221538.0,Pop
3,5,7 rings,1,2019-03-16,Ariana Grande,https://open.spotify.com/track/6ocbgoVGwYJhOv1...,Indonesia,125837.0,0.778,0.317,1.0,-10.732,0.3340,0.59200,0.000000,0.0881,0.3270,140.048,178627.0,Rap
4,6,Shape of You,1,2017-03-08,Ed Sheeran,https://open.spotify.com/track/7qiZfU4dY1lWllz...,Portugal,55132.0,0.825,0.652,1.0,-3.183,0.0802,0.58100,0.000000,0.0931,0.9310,95.977,233713.0,R&B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9171,15034,Die For You (feat. Dominic Fike),14,2021-03-19,Justin Bieber,https://open.spotify.com/track/6Z2pe32hFnUd2x5...,United States,1012774.0,0.729,0.632,1.0,-6.572,0.0320,0.00196,0.000003,0.0848,0.7020,135.027,198426.0,Pop
9172,15036,Tarabin Taraba,14,2021-01-20,Bo9al,https://open.spotify.com/track/0rQWThz77TngKHq...,Morocco,6453.0,0.881,0.797,9.0,-5.466,0.2640,0.18600,0.000000,0.1040,0.7730,139.879,140571.0,Rap
9173,15037,To Eikosalepto,14,2019-09-16,Bloody Hawk,https://open.spotify.com/track/1gxli3nPyZZyeo8...,Greece,8357.0,0.789,0.670,8.0,-8.229,0.4290,0.50100,0.000000,0.0785,0.4070,90.328,322927.0,Rap
9174,15043,Addict Of Magic,14,2021-06-25,Picture This,https://open.spotify.com/track/0deLQzMIEw6l3pW...,Ireland,31208.0,0.739,0.470,0.0,-7.555,0.0269,0.58400,0.000000,0.1530,0.6030,88.017,166897.0,Pop


In [63]:
#Saving the sub dataset for testing.
from pathlib import Path
path = Path('../data/cleanedDataSetWithGenres.csv')
path.parent.mkdir(parents=True, exist_ok=True)
df_with_genre.to_csv(path)